In [1]:
# import packages
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
import datetime
import functions_hiwi as fct
import warnings
import xarray as xr
from matplotlib.patches import Circle
from matplotlib.patches import Rectangle
import geopandas as gpd
import regionmask

In [2]:
# set ignore warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)

In [3]:
# set main directory
main_dir = r'C:\Users\illic\Python\data_hiwi\Emschergenossenschaft'

In [4]:
# set path to data
emscher_path = os.path.join(main_dir, r'EGLV_PWS_dataset_5min_v03.nc')
emscher_ref_1_path = os.path.join(main_dir, r'EGLV_refenzdaten\EGLV_rs_1min.nc')
emscher_ref_5_path = os.path.join(main_dir, r'EGLV_refenzdaten\EGLV_rs_5min2.nc')
emscher_ref_60_path = os.path.join(main_dir, r'EGLV_refenzdaten\EGLV_rs_60min.nc')

In [5]:
# read data
ds_emscher = xr.open_dataset(emscher_path)
ds_emscher_ref_1 = xr.open_dataset(emscher_ref_1_path)
ds_emscher_ref_5 = xr.open_dataset(emscher_ref_5_path)
ds_emscher_ref_60 = xr.open_dataset(emscher_ref_60_path)

In [6]:
# id neu setzen
id = np.arange(1, len(ds_emscher.id) + 1, 1) # oder: id = ['id_' + str(id) for id in (np.arange(1, len(ds_emscher.id) + 1, 1))]
ds_emscher['id'] = id
id = np.arange(1, len(ds_emscher_ref_1.id) + 1, 1)
ds_emscher_ref_1['id'] = id
ds_emscher_ref_5['id'] = id
ds_emscher_ref_60['id'] = id

In [7]:
# create DataFrame
df_emscher = pd.DataFrame(data=ds_emscher.rainfall.sel(id=slice(1, 10)).values.T, index=ds_emscher.time.values ,columns=ds_emscher.id.sel(id=slice(1, 10)).values)

# create DataFrame with no NaN values
df_emscher_nonan = df_emscher.fillna(0)

In [8]:
# create DataFrame with coordinates
df_coordinaten_emscher = pd.DataFrame(data=(ds_emscher.lon.values, ds_emscher.lat.values), index=['lon', 'lat'], columns=ds_emscher.id.values).T
coordinates_primary_utm32 = pd.DataFrame(data=(ds_emscher_ref_1.x.values, ds_emscher_ref_1.y.values), index=['lon', 'lat'], columns=ds_emscher_ref_1.id.values).T

In [9]:
# transform coordinates to x, y

# x, y = fct.LatLon_To_XY(primary_locations_df['lon'], primary_locations_df['lat'])
# coordinates_primary_utm32 = pd.DataFrame({'lon': x, 'lat': y}, index=primary_locations_df.index)

x, y = fct.LatLon_To_XY(df_coordinaten_emscher['lon'], df_coordinaten_emscher['lat'])
coordinates_secondary_utm32 = pd.DataFrame({'lon': x, 'lat': y}, index=df_coordinaten_emscher.index)

In [10]:
# read shapefile
shapefile_path = r'C:\Users\illic\Python\data_hiwi\Emschergenossenschaft\Shapefiles\eglv_grenze_utm.shp'
shapefile = gpd.read_file(shapefile_path)

In [11]:
# # set negative values of df to NaN
# cond_1min = PluvioDataRT_ds.rainfall < 0 # True wenn condition erfüllt
# cond_1h = PluvioDataRT_1h_ds.rainfall < 0

# PluvioDataRT_ds = PluvioDataRT_ds.where(~cond_1min) # da wo True, also condition erfüllt ist, wird NaN gesetzt
# PluvioDataRT_1h_ds = PluvioDataRT_1h_ds.where(~cond_1h)

In [12]:
# # create DataFrame
# PluvioDataRT_df = pd.DataFrame(data=PluvioDataRT_ds.rainfall.values.T, index=PluvioDataRT_ds.time.values ,columns=PluvioDataRT_ds.id.values)
# PluvioDataRT_1h_df = pd.DataFrame(data=PluvioDataRT_1h_ds.rainfall.values, index=PluvioDataRT_1h_ds.time.values ,columns=PluvioDataRT_1h_ds.id.values)

In [37]:
# Definition der Region als Liste von Polygonen (hier nur ein Polygon)
regions = regionmask.Regions([
    [[10, 50], [20, 50], [20, 60], [10, 60]]
])

# Benennung der Region
regions.name = "Beispielregion"

In [38]:
regions

<regionmask.Regions 'Beispielregion'>
overlap:  None

Regions:
0 r0 Region0

[1 regions]

In [13]:
shapefile

,Id,NAME,CNT_NAME,AREA,PERIMETER,geometry
0,2,Emschergenossenschaft,0,863.590505,0.0,"POLYGON ((340451.718 5715143.124, 340595.119 5..."
1,0,Lippeverband,0,3309.934617,0.0,"POLYGON ((340451.718 5715143.124, 340396.407 5..."


In [32]:
# shapefile.plot()

In [15]:
poly_emscher = regionmask.Regions(list(shapefile.geometry))

In [41]:
poly_emscher

<regionmask.Regions 'unnamed'>
overlap:  None

Regions:
0 r0 Region0
1 r1 Region1

[2 regions]

In [34]:
mask = poly_emscher.mask(ds_emscher)

c:\Users\illic\miniforge3\envs\hiwi_emscher\lib\site-packages\regionmask\core\_deprecate.py:108: UserWarning: No gridpoint belongs to any region. Returning an all-NaN mask.
  return func(*args, **kwargs)


In [35]:
mask

<xarray.DataArray 'mask' (id: 3550)> Size: 28kB
array([nan, nan, nan, ..., nan, nan, nan])
Coordinates:
  * id           (id) int64 28kB 1 2 3 4 5 6 7 ... 3545 3546 3547 3548 3549 3550
    elevation    (id) float64 28kB 32.0 31.0 28.0 30.0 ... 95.0 83.0 114.0 200.0
    lon          (id) float64 28kB 6.549 6.527 6.536 6.546 ... 8.434 8.638 8.876
    lat          (id) float64 28kB 51.43 51.44 51.43 51.44 ... 51.69 51.85 51.9
    mac_address  (id) <U17 241kB '70:ee:50:58:c1:02' ... '70:ee:50:00:1f:a4'
Attributes:
    standard_name:  region
    flag_values:    []
    flag_meanings:

In [19]:
ds_emscher_mask = ds_emscher.id.where(mask==0)

In [20]:
ds_emscher_mask

<xarray.DataArray 'id' (id: 3550)> Size: 28kB
array([nan, nan, nan, ..., nan, nan, nan])
Coordinates:
  * id           (id) int64 28kB 1 2 3 4 5 6 7 ... 3545 3546 3547 3548 3549 3550
    elevation    (id) float64 28kB 32.0 31.0 28.0 30.0 ... 95.0 83.0 114.0 200.0
    lon          (id) float64 28kB 6.549 6.527 6.536 6.546 ... 8.434 8.638 8.876
    lat          (id) float64 28kB 51.43 51.44 51.43 51.44 ... 51.69 51.85 51.9
    mac_address  (id) <U17 241kB '70:ee:50:58:c1:02' ... '70:ee:50:00:1f:a4'